## Understading HGDL's Arguments
First, let's look at its docstring

In [1]:
from hgdl.hgdl import HGDL

In [2]:
HGDL?

Init signature: HGDL(*args, **kwargs)
Docstring:     
HGDL
    * Hybrid - uses both local and global optimization
    * G - uses global optimizer
    * D - uses deflation
    * L - uses local extremum localMethod
    Mandatory Parameters:
        * func - should return a scalar given a numpy array x
        -- note: use functools.partial if you have optional params
        * grad - gradient vector at x
        * bounds - numpy array of bounds in same format as scipy.optimize
    Optional Parameters:
        * Overall Parameters -----------------------------------
        * hess (None) - hessian array at x - you may need this depending on the local method
        * client - (None->HGDL initializes if None) dask.distributed.Client object
            -- this lets you interface with clusters via dask with Client(myCluster)
        * num_epochs (10) - the number of epochs. 1 epoch is 1 global step + 1 local run
        * numWorkers (logical cpu cores -1) - how many processes to use
        

* Overall Parameters -----------------------------------
       * These are parameters that control the overall behavior
    * hess (None) - hessian array at x - you may need this depending on the local method
        * This has the same format as grad in the mandatory arguments. This should return the 
            hessian at a single point x
    * client - (None->HGDL initializes if None) dask.distributed.Client object
        * on a large computer, you could want to do something similar to 
            https://docs.dask.org/en/latest/setup/hpc.html
    * num_epochs (10) - the number of epochs. 1 epoch is 1 global step + 1 local run
        * This sets the number of epochs run. There are a few layers HGDL:
            - for each epoch
                - global step: 
                    for num_individuals:
                        calculate new points to probe and evaluate func at each
                - local step:
                    for max_local steps
                        for num_individuals:
                            run local method
    * fix_rng (True) - sets random numbers to be fixed (for reproducibility)
        * HGDL provides a random number generator to all methods. This is seeded with 42 unless
            told otherwise. the reason for this is for replicating the code in our paper
    * bestX (5) - maximum number of minima and global results to put in get_final()
        * This is the maximum length of minima_x and global_x in the dictionary returned by 
            get_final(). set to -1 for all. 
    * num_individuals (25) - the number of individuals to run for both global and local methods
    * x0 (None) starting points to probe
        * this should be starting point in the form (num_samples, dimension_of_space)
    * global_method ('genetic') - these control what global and local methods
        * I have build in 'genetic' and Marcus's 'gaussian'. to add more, just look in the
            git repo under global, and add to run_global in the same style as exists
    * local_method ('my_newton') -    are used by HGDL
        * same but for local methods. i have 'scipy' and 'my_newton', but again, you can just
            add to the local folder

* Global Method Parameters -----------------------------
    * global_args ((,)) - arguments to global method
    * global_kwargs({}) - kwargs for global method
        - note: these let you pass custom info to your method of choice
        - When you make a custom global method, you could have additional parameters or kwargs. these give you a way of storing them


* Deflation Parameters ---------------------------------
    * alpha (0.1) - the alpha term of the bump function
        * alpha determines the shape of the bump function used to 
            deflate the minima
    * r (.3) - the radius for the bump function
        * r determines how far away something is for it to be in
            range of deflation. it is the range of the bump function


* Local Method Parameters ------------------------------
    * local_args ((,)) - arguments to global method
    * local_kwargs({}) - kwargs for global method
        -- note: these let you pass custom info to your method of choice
        * same as with global_args and global_kwargs
            with the 'scipy' option you can pass options to scipy
            such as tolerances
    * max_local (5) - the maximum number of local runs to do
        * the local method is run max_local*num_individual times